In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import random
import torch
import torch.nn as nn
from custommodels import LoadDataset, ResNet50, DenseNet121, MobileNetV2
from training import ModelTrainer

In [2]:
# to reproduce
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
db = "HAM"
root = f"../../dataset/CAI 2025/{db}/"
df_train = pd.read_csv(f"{root}/dataframe/df_distance_train.csv")
df_valid = pd.read_csv(f"{root}/dataframe/df_distance_valid.csv")
df_test = pd.read_csv(f"{root}/dataframe/df_distance_test.csv")

loader = LoadDataset(label="label", batch_size=16)
train_loader, valid_loader, test_loader = loader.create_dataloaders(df_train, df_valid, df_test)

Width: 200 Height: 150


# DenseNet

In [4]:
models = ["RES", "DENSE", "MOBILE"]

model_save_directory = f"{root}model/{models[1]}/"
if not os.path.exists(model_save_directory):
    os.makedirs(model_save_directory)
    print(f"{model_save_directory} created.")
else:
    print(f"{model_save_directory} aready exist.")
        
df_valid_filepath = f"{root}dataframe/df_valid_{models[1]}.csv"
df_test_filepath = f"{root}dataframe/df_test_{models[1]}.csv"

../../dataset/CAI 2025/HAM/model/DENSE/ created.


In [8]:
num_class = 8
num_epochs = 30
lr = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet121(num_class).to(device)
trainer = ModelTrainer()
best_val_file = trainer.train(model, train_loader, valid_loader, model_save_directory, num_epochs=num_epochs, lr=lr)

Outputs shape: torch.Size([16, 8])
Outputs: tensor([[ 0.6103, -0.0879, -0.2834, -0.3834,  1.2911, -0.5435, -0.1924,  0.0657],
        [ 0.2813, -0.3094, -0.6580,  0.2825,  0.4217, -0.6519,  0.0356, -0.4843],
        [-0.1475, -1.0301,  0.2355, -0.3393,  1.1840, -0.3170, -1.3201,  0.6693],
        [ 0.4064, -0.1580, -0.6222, -0.2738,  0.8584, -0.2091,  0.1552, -0.1299],
        [-0.0466, -0.1066, -0.3220,  0.1892,  0.5468, -0.2143,  0.0230,  0.6249],
        [ 0.5081, -0.1314, -0.3043,  0.2236,  0.0990, -0.0171, -0.6231,  0.4407],
        [ 0.0765, -0.5912, -0.5073,  0.1639,  0.3920, -0.1186,  0.1073, -0.3738],
        [-0.0264,  0.2046, -0.3678,  0.1322,  0.5219, -0.3537, -0.3272, -0.1436],
        [ 0.1615, -0.7940, -0.5219,  0.2894,  1.3402, -0.4593, -0.5330, -0.3476],
        [ 0.6707, -0.4145,  0.3201, -0.4594,  0.7444, -0.6188, -0.8979, -0.0272],
        [ 0.2084,  0.2695, -0.1218,  0.1832,  0.7997, -0.2916, -0.0970,  0.0632],
        [ 0.7139, -0.0310,  0.3488, -0.6736,  0.6798, 

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [ ]:
model = ResNet50(num_class).to(device)
model.load_state_dict(torch.load(best_val_file))
test_predictions, test_outputs = trainer.evaluate(model, valid_loader)
df_valid["pred"] = test_predictions
df_valid["proba"] = test_outputs
df_valid.to_csv(df_valid_filepath, index=False)

In [ ]:
test_predictions, test_outputs = trainer.evaluate(model, test_loader)
df_test["pred"] = test_predictions
df_test["proba"] = test_outputs
df_test.to_csv(df_valid_filepath, index=False)